In [1]:
import sys
import os
import shutil
import datetime
import re
import pandas as pd
from collections import Counter
#sys.path.append(os.path.abspath("/home/jose/Dropbox/Doktorarbeit/"))
#from reading_robot import load_data, tei2text, text2features, classify, process_labels, describe_data,feature_analysis, cull_data, st_tests, metadata2numbers, cluster, call_stylo, prepare_subcorpus
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
%matplotlib inline
from shutil import copyfile
import numpy as np
import glob

# Read Data

In [2]:
wdir = "../data/prosa-SdO/only-tokens/directorio_base/"

In [3]:
files = glob.glob(wdir + "/*.txt")


In [4]:
files

['../data/prosa-SdO/only-tokens/directorio_base\\Horozco_La-famosa-historia-de-Ruth.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Horozco_Relaciones-Históricas-Toledanas.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Mejía_Carlos-V.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Mejía_Coloquio-del-Convite.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Mejía_Coloquio-del-Porfiado.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Mejía_Coloquio-del-Sol.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Mejía_Dialogo-de-la-Tierra.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Mejía_Diаlogo-de-los-Médicos.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Mejía_Diаlogo-Natural.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Mejía_Silva-de-Varia-Lección.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Valdés-Alfonso-de_Diаlogo-de-las-cosas-acaecidas-en-Roma.txt',
 '../data/prosa-SdO/only-tokens/directorio_base\\Va

In [5]:
files_df = pd.DataFrame(files, columns=["path"])

In [6]:
files_df

,path
0,../data/prosa-SdO/only-tokens/directorio_base\...
1,../data/prosa-SdO/only-tokens/directorio_base\...
2,../data/prosa-SdO/only-tokens/directorio_base\...
3,../data/prosa-SdO/only-tokens/directorio_base\...
4,../data/prosa-SdO/only-tokens/directorio_base\...
5,../data/prosa-SdO/only-tokens/directorio_base\...
6,../data/prosa-SdO/only-tokens/directorio_base\...
7,../data/prosa-SdO/only-tokens/directorio_base\...
8,../data/prosa-SdO/only-tokens/directorio_base\...
9,../data/prosa-SdO/only-tokens/directorio_base\...


In [7]:
files_df["file_name"] = files_df["path"].str.findall("\\\(.*?)\.txt").str[0]

In [8]:
files_df

,path,file_name
0,../data/prosa-SdO/only-tokens/directorio_base\...,Horozco_La-famosa-historia-de-Ruth
1,../data/prosa-SdO/only-tokens/directorio_base\...,Horozco_Relaciones-Históricas-Toledanas
2,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Carlos-V
3,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Convite
4,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Porfiado
5,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Sol
6,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Dialogo-de-la-Tierra
7,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Diаlogo-de-los-Médicos
8,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Diаlogo-Natural
9,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Silva-de-Varia-Lección


In [9]:
files_df["author"] = files_df["file_name"].str.findall("^(.*?)_").str[0]

In [10]:
files_df["text_title"] = files_df["file_name"].str.findall("_(.*?)$").str[0]

In [13]:
files_df

,path,file_name,author,text_title
0,../data/prosa-SdO/only-tokens/directorio_base\...,Horozco_La-famosa-historia-de-Ruth,Horozco,La-famosa-historia-de-Ruth
1,../data/prosa-SdO/only-tokens/directorio_base\...,Horozco_Relaciones-Históricas-Toledanas,Horozco,Relaciones-Históricas-Toledanas
2,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Carlos-V,Mejía,Carlos-V
3,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Convite,Mejía,Coloquio-del-Convite
4,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Porfiado,Mejía,Coloquio-del-Porfiado
5,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Sol,Mejía,Coloquio-del-Sol
6,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Dialogo-de-la-Tierra,Mejía,Dialogo-de-la-Tierra
7,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Diаlogo-de-los-Médicos,Mejía,Diаlogo-de-los-Médicos
8,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Diаlogo-Natural,Mejía,Diаlogo-Natural
9,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Silva-de-Varia-Lección,Mejía,Silva-de-Varia-Lección


In [38]:
files_df.author.replace("Valdés-Juan-de", "Valdés-Juan", inplace=True)
files_df.author.replace("Valdés-Juan", "Valdés-Juan", inplace=True)


In [41]:
files_df.author.replace("Valdés-Alfonso-de", "Valdés-Alfonso", inplace=True)
files_df.author.replace("Valdés-Alfonso", "Valdés-Alfonso", inplace=True)



In [42]:
files_df.author.value_counts()

Vives-Juan-Luis    9
Mejía             8
Valdés-Juan        3
Valdés-Alfonso     2
Horozco            2
Name: author, dtype: int64

In [43]:
files_df

,path,file_name,author,text_title,text,tokens_sum
file_name,,,,,,
Horozco_La-famosa-historia-de-Ruth,../data/prosa-SdO/only-tokens/directorio_base\...,Horozco_La-famosa-historia-de-Ruth,Horozco,La-famosa-historia-de-Ruth,\nlas\npersonas\nque\nen\nella\nse\nintroduzen...,3730
Horozco_Relaciones-Históricas-Toledanas,../data/prosa-SdO/only-tokens/directorio_base\...,Horozco_Relaciones-Históricas-Toledanas,Horozco,Relaciones-Históricas-Toledanas,\nes\nde\nsaber\nque\nen\nesta\nçibdad\nde\nto...,78506
Mejía_Carlos-V,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Carlos-V,Mejía,Carlos-V,\ncomíença\nla\nuida\ne\nystoria\nde\nel\nymví...,207130
Mejía_Coloquio-del-Convite,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Convite,Mejía,Coloquio-del-Convite,\nlos\ndos\ncoloquios\nde\nel\nconvite\nde\nlo...,13629
Mejía_Coloquio-del-Porfiado,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Porfiado,Mejía,Coloquio-del-Porfiado,\ncoloquio\nde\nel\nporfiado\nen\nel\ncual\nin...,8222
Mejía_Coloquio-del-Sol,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Sol,Mejía,Coloquio-del-Sol,\ncoloquio\nde\nel\nsol\nen\nel\ncual\nse\ntra...,4184
Mejía_Dialogo-de-la-Tierra,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Dialogo-de-la-Tierra,Mejía,Dialogo-de-la-Tierra,\ndiálogo\nde\nla\ntierra\nen\nel\ncual\npor\n...,3231
Mejía_Diаlogo-de-los-Médicos,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Diаlogo-de-los-Médicos,Mejía,Diаlogo-de-los-Médicos,\ndiálogo\nprimero\nde\nlos\nmédicos\nel\nargu...,12105
Mejía_Diаlogo-Natural,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Diаlogo-Natural,Mejía,Diаlogo-Natural,\ndiálogo\nnatural\nen\nel\ncual\nbreve\ny\nar...,5679


In [44]:
for index, row in files_df.iterrows():
    with open(row["path"], "r", errors="replace", encoding="utf-8") as fin:
        content_str = fin.read()
    files_df.loc[index, "text"] = content_str

In [45]:
files_df

,path,file_name,author,text_title,text,tokens_sum
file_name,,,,,,
Horozco_La-famosa-historia-de-Ruth,../data/prosa-SdO/only-tokens/directorio_base\...,Horozco_La-famosa-historia-de-Ruth,Horozco,La-famosa-historia-de-Ruth,\nlas\npersonas\nque\nen\nella\nse\nintroduzen...,3730
Horozco_Relaciones-Históricas-Toledanas,../data/prosa-SdO/only-tokens/directorio_base\...,Horozco_Relaciones-Históricas-Toledanas,Horozco,Relaciones-Históricas-Toledanas,\nes\nde\nsaber\nque\nen\nesta\nçibdad\nde\nto...,78506
Mejía_Carlos-V,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Carlos-V,Mejía,Carlos-V,\ncomíença\nla\nuida\ne\nystoria\nde\nel\nymví...,207130
Mejía_Coloquio-del-Convite,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Convite,Mejía,Coloquio-del-Convite,\nlos\ndos\ncoloquios\nde\nel\nconvite\nde\nlo...,13629
Mejía_Coloquio-del-Porfiado,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Porfiado,Mejía,Coloquio-del-Porfiado,\ncoloquio\nde\nel\nporfiado\nen\nel\ncual\nin...,8222
Mejía_Coloquio-del-Sol,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Sol,Mejía,Coloquio-del-Sol,\ncoloquio\nde\nel\nsol\nen\nel\ncual\nse\ntra...,4184
Mejía_Dialogo-de-la-Tierra,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Dialogo-de-la-Tierra,Mejía,Dialogo-de-la-Tierra,\ndiálogo\nde\nla\ntierra\nen\nel\ncual\npor\n...,3231
Mejía_Diаlogo-de-los-Médicos,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Diаlogo-de-los-Médicos,Mejía,Diаlogo-de-los-Médicos,\ndiálogo\nprimero\nde\nlos\nmédicos\nel\nargu...,12105
Mejía_Diаlogo-Natural,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Diаlogo-Natural,Mejía,Diаlogo-Natural,\ndiálogo\nnatural\nen\nel\ncual\nbreve\ny\nar...,5679


In [46]:
files_df.index = files_df.file_name

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
reg_ex = r"(?u)\b[\w_@]+\b|[¶\(»\]\?\.\–\!’•\|“\>\)\-\—\:\}\*\&…¿\/=¡\"\'·+\{\#\[;­,«~]"
sort_columns_by_frequency = True
lowercase = True
max_features = 1000000
analyzer = "word" #{‘word’, ‘char’, ‘char_wb’}
ngram_range = (1,1) #ngram_range = None

#vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(corpus)

#vectorizer = CountVectorizer()

vec = CountVectorizer(
        token_pattern = reg_ex,
        lowercase = lowercase,
        max_features = max_features,
        analyzer = analyzer,
        ngram_range = ngram_range,
        )

tokens = vec.fit_transform(files_df.text.tolist())

tokens_df = pd.DataFrame(tokens.toarray(), columns = vec.get_feature_names(), index = files_df["file_name"])

In [49]:
tokens_df

,',",",-,.,_diez,_dos,_nueve,_ocho,_siete,_veinte,...,úsanse,útero,útil,útiles,útilmente,úvose,ûperbolikñj,ûpostasij,üsanloe,ÿxij
file_name,,,,,,,,,,,,,,,,,,,,,
Horozco_La-famosa-historia-de-Ruth,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Horozco_Relaciones-Históricas-Toledanas,0,33,1,1,13,4,1,1,1,13,...,0,0,1,0,0,0,0,0,0,0
Mejía_Carlos-V,1,0,19,4,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
Mejía_Coloquio-del-Convite,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Mejía_Coloquio-del-Porfiado,0,0,0,2,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0
Mejía_Coloquio-del-Sol,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Mejía_Dialogo-de-la-Tierra,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Mejía_Diаlogo-de-los-Médicos,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
Mejía_Diаlogo-Natural,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
files_df["tokens_sum"] = tokens_df.sum(axis=1)

In [51]:
files_df

,path,file_name,author,text_title,text,tokens_sum
file_name,,,,,,
Horozco_La-famosa-historia-de-Ruth,../data/prosa-SdO/only-tokens/directorio_base\...,Horozco_La-famosa-historia-de-Ruth,Horozco,La-famosa-historia-de-Ruth,\nlas\npersonas\nque\nen\nella\nse\nintroduzen...,3730
Horozco_Relaciones-Históricas-Toledanas,../data/prosa-SdO/only-tokens/directorio_base\...,Horozco_Relaciones-Históricas-Toledanas,Horozco,Relaciones-Históricas-Toledanas,\nes\nde\nsaber\nque\nen\nesta\nçibdad\nde\nto...,78506
Mejía_Carlos-V,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Carlos-V,Mejía,Carlos-V,\ncomíença\nla\nuida\ne\nystoria\nde\nel\nymví...,207130
Mejía_Coloquio-del-Convite,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Convite,Mejía,Coloquio-del-Convite,\nlos\ndos\ncoloquios\nde\nel\nconvite\nde\nlo...,13629
Mejía_Coloquio-del-Porfiado,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Porfiado,Mejía,Coloquio-del-Porfiado,\ncoloquio\nde\nel\nporfiado\nen\nel\ncual\nin...,8222
Mejía_Coloquio-del-Sol,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Coloquio-del-Sol,Mejía,Coloquio-del-Sol,\ncoloquio\nde\nel\nsol\nen\nel\ncual\nse\ntra...,4184
Mejía_Dialogo-de-la-Tierra,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Dialogo-de-la-Tierra,Mejía,Dialogo-de-la-Tierra,\ndiálogo\nde\nla\ntierra\nen\nel\ncual\npor\n...,3231
Mejía_Diаlogo-de-los-Médicos,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Diаlogo-de-los-Médicos,Mejía,Diаlogo-de-los-Médicos,\ndiálogo\nprimero\nde\nlos\nmédicos\nel\nargu...,12105
Mejía_Diаlogo-Natural,../data/prosa-SdO/only-tokens/directorio_base\...,Mejía_Diаlogo-Natural,Mejía,Diаlogo-Natural,\ndiálogo\nnatural\nen\nel\ncual\nbreve\ny\nar...,5679


In [52]:
files_df.author.value_counts().mean()

4.8

In [53]:
files_df.shape[0]

24

In [54]:
files_df.author.value_counts().index

Index(['Vives-Juan-Luis', 'Mejía', 'Valdés-Juan', 'Valdés-Alfonso',
       'Horozco'],
      dtype='object')

In [55]:
files_df.to_csv("./../data/prosa-SdO/metadata.tsv", sep="\t")

In [56]:
number_texts = files_df.shape[0]
number_authors = len(files_df.author.unique())
mean_number_text_author = files_df.author.value_counts().mean()
std_number_text_author = files_df.author.value_counts().std()
mean_length_text = files_df.tokens_sum.mean()
std_length_text = files_df.tokens_sum.std()
distribution_authors_texts = files_df.author.value_counts()


In [57]:
summary_corpus_df = pd.DataFrame(
            [number_texts,   number_authors,  mean_number_text_author,   std_number_text_author,  mean_length_text, std_length_text, distribution_authors_texts  ],
    index=["number_texts","number_authors","mean_number_text_author", "std_number_text_author","mean_length","std_length", "distribution_authors_texts"  ],
    columns = ["16-17-prosa"]).T

In [58]:
summary_corpus_df

,number_texts,number_authors,mean_number_text_author,std_number_text_author,mean_length,std_length,distribution_authors_texts
16-17-prosa,24,5,4.8,3.42053,65924.9,75740.6,Vives-Juan-Luis 9 Mejía 8 Vald...


In [59]:
summary_corpus_df.to_csv("./../data/prosa-SdO/summary.tsv", sep="\t")